In [5]:
import pandas as pd
import os 
file = "/data/steam_games.csv"
file = os.getcwd() + file
#raw_steam_data = pd.read_csv(file, nrows=8000)
raw_steam_data = pd.read_csv(file)

In [6]:
raw_steam_data['name'].size #40833 games
names = raw_steam_data['name'] 
names.size #40833 names (possibly not unique) 
types = raw_steam_data['types']
#types[types.duplicated(keep=False)].size

#example of duplicate names -> unsuitable for unique id
for i in range(0, raw_steam_data['name'].size):
    if "The Deer" == raw_steam_data['name'][i]:
        print(raw_steam_data.iloc[i])
        print(i)

url                         https://store.steampowered.com/app/400240/The_...
types                                                                     app
name                                                                 The Deer
desc_snippet                "The Deer" is an educational inter­ac­tive exp...
recent_reviews                                                            NaN
all_reviews                 Mostly Positive,(317),- 70% of the 317 user re...
release_date                                                      Dec 2, 2015
developer                                        Fabio Ferrara,Giulia Airoldi
publisher                                           Chubby Pixel,Chubby Pixel
popular_tags                Indie,Adventure,Education,Family Friendly,Shor...
game_details                Single-player,Full controller support,Steam Tr...
languages                                                             English
achievements                                                    

In [7]:
#remove all non apps from the dataframe
for i in range(0, raw_steam_data['types'].size):
    if raw_steam_data['types'][i] != 'app':
        #remove the row for that index
        #this means if index 5 is removed there is no longer and index 5 in the data frame going stright from 4 to 6
        raw_steam_data = raw_steam_data.drop([i])

In [8]:
#reset index in the dataframe so there is no longer missing indexes
# and then reduce the data frame size to 8000

#create a range
myrange = range(0, 8000)
#rest index so there is no longer missing indexes
raw_steam_data.reset_index(drop=True, inplace=True)
#reduce size of data frame to 8000
raw_steam_data = raw_steam_data[:8000]

In [9]:
#purging unwanted data
purged_steam_data = raw_steam_data.drop(columns=['types', 'desc_snippet', 'achievements','game_description', 'minimum_requirements', 'recommended_requirements'])
purged_steam_data.iloc[5]

url               https://store.steampowered.com/app/601150/Devi...
name                                                Devil May Cry 5
recent_reviews    Very Positive,(408),- 87% of the 408 user revi...
all_reviews       Very Positive,(9,645),- 92% of the 9,645 user ...
release_date                                            Mar 7, 2019
developer                                          CAPCOM Co., Ltd.
publisher                         CAPCOM Co., Ltd.,CAPCOM Co., Ltd.
popular_tags      Action,Hack and Slash,Great Soundtrack,Demons,...
game_details      Single-player,Online Multi-Player,Online Co-op...
languages         English,French,Italian,German,Spanish - Spain,...
genre                                                        Action
mature_content     Mature Content Description  The developers de...
original_price                                               $59.99
discount_price                                               $70.42
Name: 5, dtype: object

In [10]:
import re
import math
amount_p = re.compile(r'\d+(,\d+)*')
value_p = re.compile(r'\d+')

def process_review(review):
    #print(review)
    if isinstance(review, str) :
        str_num = amount_p.search(review).group()
        nums = value_p.findall(str_num)
        amt = 0
        if 1 < len(nums):
            for i in range(0, len(nums) - 1):
                prev = nums[i + 1]
                power = len(prev)
                prev = int(prev)
                amt += int(nums[i])*(10**power) + prev
                
        else:
            amt = nums[0]
        
        all_nums = value_p.findall(review)
        if 2 < len(all_nums):
            val = all_nums[len(nums)]
        else:
            val = -1
            
        return [amt, val]
    else: 
        return []
    
def get_element(nums, index):
    if index < len(nums):
        return int(nums[index])
    else:
        return 0

recent_review_numbers = purged_steam_data.apply(lambda x: process_review(x['recent_reviews']), axis=1)
recent_review_numbers


purged_steam_data['amt_rec_revs'] = recent_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['rec_rev_score'] = recent_review_numbers.apply(lambda x: get_element(x, 1))

alltime_review_numbers = purged_steam_data.apply(lambda x: process_review(x['all_reviews']), axis=1)

purged_steam_data['amt_all_revs'] = alltime_review_numbers.apply(lambda x: get_element(x, 0))
purged_steam_data['all_rev_score'] = alltime_review_numbers.apply(lambda x: get_element(x, 1))

purged_steam_data = purged_steam_data.drop(columns=['recent_reviews','all_reviews'])

#keys for new columns:
#'amt_rec_revs' -> the amount of recent reviews
#'rec_rev_score' -> the percentage of the recent reviews
#'amt_all_revs' -> the amount of all reviews
#'all_rev_score' - > the perceent of all reviews


In [11]:
import functools 

def handle_prices(price):
    if isinstance(price, str):
        prices = value_p.findall(price)
        if len(prices):
            price = float(prices[0]) 
            if 1 < len(prices):
                price += float(prices[1])/100
            return price
        else:
            return 0
    else:
        return -1 #return nan

purged_steam_data['original_price'] = purged_steam_data.apply(lambda x: handle_prices(x['original_price']), axis=1)
purged_steam_data['discount_price'] = purged_steam_data.apply(lambda x: handle_prices(x['discount_price']), axis=1)


#value_p.findall(purged_steam_data['original_price'][4])

In [12]:
purged_steam_data

url  \
0       https://store.steampowered.com/app/379720/DOOM/   
1     https://store.steampowered.com/app/578080/PLAY...   
2     https://store.steampowered.com/app/637090/BATT...   
3       https://store.steampowered.com/app/221100/DayZ/   
4     https://store.steampowered.com/app/8500/EVE_On...   
...                                                 ...   
7995  https://store.steampowered.com/app/391040/F1_2...   
7996  https://store.steampowered.com/app/208500/F1_2...   
7997  https://store.steampowered.com/app/40930/The_M...   
7998  https://store.steampowered.com/app/360650/12_L...   
7999  https://store.steampowered.com/app/808080/Ulti...   

                                        name  release_date  \
0                                       DOOM  May 12, 2016   
1              PLAYERUNKNOWN'S BATTLEGROUNDS  Dec 21, 2017   
2                                 BATTLETECH  Apr 24, 2018   
3                                       DayZ  Dec 13, 2018   
4                                 EVE Online   May 6, 2003   
...                                      ...           ...   
7995                                 F1 2016  Aug 18, 2016   
7996                                F1 2012™  Sep 17, 2012   
7997  The Misadventures of P.B. Winterbottom  Apr 20, 2010   
7998  12 Labours of Hercules III: Girl Power   Aug 7, 2015   
7999                      Ultimate Racing 2D  May 24, 2018   

                                developer  \
0                             id Software   
1                        PUBG Corporation   
2                     Harebrained Schemes   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters,Feral Interactive (Mac)   
7996               Codemasters Birmingham   
7997                    The Odd Gentlemen   
7998       Jetdogs Studios,Zoom Out Games   
7999                          Applimazing   

                                              publisher  \
0                 Bethesda Softworks,Bethesda Softworks   
1                     PUBG Corporation,PUBG Corporation   
2               Paradox Interactive,Paradox Interactive   
3               Bohemia Interactive,Bohemia Interactive   
4                                               CCP,CCP   
...                                                 ...   
7995  Codemasters,Feral Interactive (Mac),Codemaster...   
7996                            Codemasters,Codemasters   
7997                                              2K,2K   
7998                    Jetdogs Studios,Jetdogs Studios   
7999                            Applimazing,Applimazing   

                                           popular_tags  \
0     FPS,Gore,Action,Demons,Shooter,First-Person,Gr...   
1     Survival,Shooter,Multiplayer,Battle Royale,PvP...   
2     Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...   
3     Survival,Zombies,Open World,Multiplayer,PvP,Ma...   
4     Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...   
...                                                 ...   
7995  Racing,Simulation,Sports,Multiplayer,Singlepla...   
7996  Racing,Simulation,Sports,Realistic,Multiplayer...   
7997  Indie,Puzzle,Platformer,Adventure,Time Manipul...   
7998  Casual,Strategy,Time Management,Singleplayer,M...   
7999         Racing,Simulation,Sports,Indie,Top-Down,2D   

                                           game_details  \
0     Single-player,Multi-player,Co-op,Steam Achieve...   
1                Multi-player,Online Multi-Player,Stats   
2     Single-player,Multi-player,Online Multi-Player...   
3     Multi-player,Online Multi-Player,Steam Worksho...   
4     Multi-player,Online Multi-Player,MMO,Co-op,Onl...   
...                                                 ...   
7995  Single-player,Multi-player,Steam Achievements,...   
7996  Single-player,Multi-player,Co-op,Steam Achieve...   
7997  Single-player,Steam Achievements,Partial Contr...   
7998  Single-player,Steam Achievements,Steam Trad

In [13]:
from collections import Counter 

def remove_duplicates(input): 
    if (type(input) == str):
        # split input string separated by space 
        input = input.split(",") 

        # joins two adjacent elements in iterable way 
        for i in range(0, len(input)): 
            input[i] = "".join(input[i]) 

        # now create dictionary using counter method 
        # which will have strings as key and their 
        # frequencies as value 
        UniqW = Counter(input) 

        # joins two adjacent elements in iterable way 
        s = " ".join(UniqW.keys()) 
        return s
    else:
        s = ""
        return s


purged_steam_data['publisher'] = purged_steam_data.apply(lambda x: remove_duplicates(x['publisher']), axis=1)

purged_steam_data

url  \
0       https://store.steampowered.com/app/379720/DOOM/   
1     https://store.steampowered.com/app/578080/PLAY...   
2     https://store.steampowered.com/app/637090/BATT...   
3       https://store.steampowered.com/app/221100/DayZ/   
4     https://store.steampowered.com/app/8500/EVE_On...   
...                                                 ...   
7995  https://store.steampowered.com/app/391040/F1_2...   
7996  https://store.steampowered.com/app/208500/F1_2...   
7997  https://store.steampowered.com/app/40930/The_M...   
7998  https://store.steampowered.com/app/360650/12_L...   
7999  https://store.steampowered.com/app/808080/Ulti...   

                                        name  release_date  \
0                                       DOOM  May 12, 2016   
1              PLAYERUNKNOWN'S BATTLEGROUNDS  Dec 21, 2017   
2                                 BATTLETECH  Apr 24, 2018   
3                                       DayZ  Dec 13, 2018   
4                                 EVE Online   May 6, 2003   
...                                      ...           ...   
7995                                 F1 2016  Aug 18, 2016   
7996                                F1 2012™  Sep 17, 2012   
7997  The Misadventures of P.B. Winterbottom  Apr 20, 2010   
7998  12 Labours of Hercules III: Girl Power   Aug 7, 2015   
7999                      Ultimate Racing 2D  May 24, 2018   

                                developer  \
0                             id Software   
1                        PUBG Corporation   
2                     Harebrained Schemes   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters,Feral Interactive (Mac)   
7996               Codemasters Birmingham   
7997                    The Odd Gentlemen   
7998       Jetdogs Studios,Zoom Out Games   
7999                          Applimazing   

                                publisher  \
0                      Bethesda Softworks   
1                        PUBG Corporation   
2                     Paradox Interactive   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters Feral Interactive (Mac)   
7996                          Codemasters   
7997                                   2K   
7998                      Jetdogs Studios   
7999                          Applimazing   

                                           popular_tags  \
0     FPS,Gore,Action,Demons,Shooter,First-Person,Gr...   
1     Survival,Shooter,Multiplayer,Battle Royale,PvP...   
2     Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...   
3     Survival,Zombies,Open World,Multiplayer,PvP,Ma...   
4     Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...   
...                                                 ...   
7995  Racing,Simulation,Sports,Multiplayer,Singlepla...   
7996  Racing,Simulation,Sports,Realistic,Multiplayer...   
7997  Indie,Puzzle,Platformer,Adventure,Time Manipul...   
7998  Casual,Strategy,Time Management,Singleplayer,M...   
7999         Racing,Simulation,Sports,Indie,Top-Down,2D   

                                           game_details  \
0     Single-player,Multi-player,Co-op,Steam Achieve...   
1                Multi-player,Online Multi-Player,Stats   
2     Single-player,Multi-player,Online Multi-Player...   
3     Multi-player,Online Multi-Player,Steam Worksho...   
4     Multi-player,Online Multi-Player,MMO,Co-op,Onl...   
...                                                 ...   
7995  Single-player,Multi-player,Steam Achievements,...   
7996  Single-player,Multi-player,Co-op,Steam Achieve...   
7997  Single-player,Steam Achievements,Partial Contr...   
7998  Single-player,Steam Achievements,Steam Trading...   
7999  Single-player,Multi-player,Online Multi-Player...   

                                              languages  \
0     English,French,Italian,German,Spa

In [14]:
#find games that have no popular tags and change to empty to string 

def remove_nans(input):
    if (isinstance(input, str)):
        return input
    else:
        return ""

purged_steam_data['popular_tags'] = purged_steam_data.apply(lambda x: remove_nans(x['popular_tags']), axis=1)

In [15]:
#find number of days since games release relative to today
import datetime
release = datetime.datetime.strptime("May 12, 2016", '%b %d, %Y')
today = datetime.datetime.now()
abs((release - today).days)

1411

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tif_matrix = vectorizer.fit_transform(purged_steam_data['popular_tags'])
for name in vectorizer.get_feature_names():
    print(name)
print(vectorizer.get_feature_names())
print(tif_matrix.shape)
#tif_matrix

1980s
1990
2d
360
3d
40k
4x
5d
6dof
abstract
access
action
addictive
adventure
agriculture
aliens
alternate
america
and
animation
anime
apocalyptic
arcade
arena
artificial
arts
assassin
asymmetric
asynchronous
atmospheric
attack
atv
audio
automation
awkward
base
baseball
based
basketball
batman
battle
beat
beautiful
benchmark
bikes
blood
bmx
board
book
bowling
builder
building
bullet
campaign
capitalism
card
cartoon
cartoony
casual
cats
character
chess
choices
choose
cinematic
city
class
classic
click
clicker
co
cold
colorful
combat
comedy
comic
competitive
conspiracy
content
control
controller
controls
conversation
crafting
crawler
crime
croft
crowdfunded
crpg
cult
customization
cute
cyberpunk
cycling
dark
dating
death
defense
demons
design
destruction
detective
development
difficult
dinosaurs
diplomacy
documentary
dog
down
dragons
drama
drawn
driving
dungeon
dungeons
dynamic
dystopian
early
economy
editing
editor
education
em
emotional
endings
epic
episodic
events
experience
experime

In [17]:
#find games that have no mature content and change to empty to string 
purged_steam_data['mature_content'] = purged_steam_data.apply(lambda x: remove_nans(x['mature_content']), axis=1)

In [18]:
from sklearn.metrics.pairwise import linear_kernel 
cosine_similarities = linear_kernel(tif_matrix, tif_matrix) 
results = {}
for idx, row in purged_steam_data.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] #get top 99 most common games
   similar_items = [(cosine_similarities[idx][i], purged_steam_data['url'][i]) for i in similar_indices] 
   results[row['url']] = similar_items[1:]

In [19]:
results[purged_steam_data['url'][1]]
#purged_steam_data['url'][0] ['https://store.steampowered.com/app/782330/DOOM_Eternal/']
#['https://store.steampowered.com/app/718560/Scythe_Digital_Edition/']

[(0.8273092328045784,
  'https://store.steampowered.com/app/813820/Realm_Royale/'),
 (0.7467227550250934,
  'https://store.steampowered.com/app/460930/Tom_Clancys_Ghost_Recon_Wildlands/'),
 (0.7460181353177834,
  'https://store.steampowered.com/app/15120/Tom_Clancys_Rainbow_Six_Vegas_2/'),
 (0.7205711683432822,
  'https://store.steampowered.com/app/555440/Deathgarden_BLOODHARVEST/'),
 (0.7064301780761303,
  'https://store.steampowered.com/app/433850/Z1_Battle_Royale/'),
 (0.6915023106754096,
  'https://store.steampowered.com/app/893520/Creative_Destruction/'),
 (0.6909868148986821,
  'https://store.steampowered.com/app/755790/Ring_of_Elysium/'),
 (0.6807123767226025,
  'https://store.steampowered.com/app/235600/Tom_Clancys_Splinter_Cell_Blacklist/'),
 (0.6634722214534972,
  'https://store.steampowered.com/app/104900/ORION_Prelude/'),
 (0.6575412628100911,
  'https://store.steampowered.com/app/215470/Primal_Carnage/'),
 (0.6548412452465646, 'https://store.steampowered.com/app/222880/Ins

In [20]:
results['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

[(0.7940682804456892,
  'https://store.steampowered.com/app/611670/The_Elder_Scrolls_V_Skyrim_VR/'),
 (0.7347128472318687,
  'https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/'),
 (0.6889144710684402,
  'https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/'),
 (0.6889144710684402,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.6260190568080446,
  'https://store.steampowered.com/app/288470/Fable_Anniversary/'),
 (0.615635607904466,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.5972939366314769,
  'https://store.steampowered.com/app/367500/Dragons_Dogma_Dark_Arisen/'),
 (0.560577428528251,
  'https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters/'),
 (0.5537780525233916, 'https://store.steampowered.com/app/551770/ECHO/'),
 (0.5363790287889696,
  'https://store.steampowered.com/app/39

In [47]:
import itertools
#cosine model function, recommends some games based of a single game
#def cos_recommend_from_game(url):
#    return results[url]


#cos_recommend('https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/', 10)

#cos_recommend: library -> an array of game url's, no_games -> the #of game recommendations
def cos_recommend(library, no_games):
    recs = {} #recommendations
    for url in library:
        for game in results[url]:
            if game[1] in recs:
                recs[game[1]] += game[0]
            else: 
                recs[game[1]] = game[0]
            
    sorted_recs = sorted(recs.items() ,key=lambda x: x[1], reverse=True)
    sorted_recs = sorted_recs[:no_games]
    
    #for rec in sorted_recs:
    #    print("game " + rec[0])
    #    print("score " + str(rec[1]))
    
    return sorted_recs
                                       
library1 = ['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

library = ['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/',
           'https://store.steampowered.com/app/813820/Realm_Royale/',
           'https://store.steampowered.com/app/55230/Saints_Row_The_Third/']

cos_recommend(library, 10)
results['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

[(0.7940682804456892,
  'https://store.steampowered.com/app/611670/The_Elder_Scrolls_V_Skyrim_VR/'),
 (0.7347128472318687,
  'https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/'),
 (0.6889144710684402,
  'https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/'),
 (0.6889144710684402,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.6260190568080446,
  'https://store.steampowered.com/app/288470/Fable_Anniversary/'),
 (0.615635607904466,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.5972939366314769,
  'https://store.steampowered.com/app/367500/Dragons_Dogma_Dark_Arisen/'),
 (0.560577428528251,
  'https://store.steampowered.com/app/204030/Fable__The_Lost_Chapters/'),
 (0.5537780525233916, 'https://store.steampowered.com/app/551770/ECHO/'),
 (0.5363790287889696,
  'https://store.steampowered.com/app/39

In [22]:
#'age' no of days since game added to store 
import dateparser
today = datetime.datetime.today()

release = dateparser.parse(purged_steam_data['release_date'][0])
age = []

for i in range(0, len(purged_steam_data)):
    str_release = purged_steam_data['release_date'][i]
    print(str_release)
    if((type(str_release) == str) and 
       (str_release != 'TBA' )):
        if(str_release.split()[0] == 'Summer'):
            year = str_release.split()[1]
            print("Jun 20, " + year)
            release = dateparser.parse("Jun 20, " + year)
            age.append(abs((release - today).days))
        else:
            release = dateparser.parse(purged_steam_data['release_date'][i])     
            age.append(abs((release - today).days))
    else:
        age.append(abs((release - today).days))
 

purged_steam_data['age'] = age
#abs((release - today).days)
purged_steam_data


May 12, 2016
Dec 21, 2017
Apr 24, 2018
Dec 13, 2018
May 6, 2003
Mar 7, 2019
Jul 22, 2016
Dec 12, 2017
May 31, 2019
Apr 19, 2018
Sep 25, 2017
May 5, 2015
Jul 27, 2017
Jul 25, 2018
Mar 16, 2017
Feb 7, 2019
Apr 5, 2018
Apr 20, 2017
Nov 9, 2010
Apr 9, 2019
May 21, 2019
Aug 31, 2016
Dec 2, 2008
Oct 10, 2007
Oct 23, 2015
Mar 27, 2018
Mar 23, 2018
Mar 28, 2016
Aug 8, 2017
Apr 18, 2016
Aug 1, 2018
Aug 21, 2017
Nov 1, 2005
Sep 2, 2008
Sep 26, 2018
Feb 7, 2012
Feb 28, 2011
Jun 13, 2019
Nov 1, 2004
Oct 13, 2014
Jan 11, 2019
Feb 16, 2006
Sep 14, 2017
Feb 15, 2016
Oct 13, 2015
Jun 5, 2019
Jan 24, 2019
Apr 2, 2015
Sep 16, 2008
Oct 9, 2017
Apr 19, 2017
Jun 6, 2012
Jun 5, 2019
Oct 27, 2010
Mar 26, 2015
Aug 15, 2018
May 31, 2017
Mar 7, 2018
Jun 5, 2018
Dec 12, 2017
Nov 15, 2006
Jan 10, 2019
May 12, 2016
Oct 18, 2018
Sep 26, 2012
Nov 2, 2017
Oct 30, 2017
Oct 8, 2015
Dec 19, 2008
Jun 11, 2019
Aug 11, 2015
Nov 30, 2015
Mar 25, 2013
May 15, 2014
Jun 25, 2012
Feb 14, 2019
May 30, 2019
Sep 19, 2018
Jun 20, 2

Oct 23, 2018
Jun 16, 2017
Aug 29, 2018
Mar 4, 2016
Jan 19, 2015
Nov 26, 2014
Aug 9, 2016
Jan 29, 2015
Nov 20, 2014
Jul 20, 2017
Dec 7, 2017
Mar 29, 2017
Feb 23, 2018
Aug 1, 2017
Dec 1, 2017
Nov 17, 2008
Apr 18, 2018
Feb 27, 2018
Feb 27, 2012
Feb 18, 2016
Feb 9, 2016
May 28, 2019
Jun 23, 2015
Dec 10, 2015
May 23, 2019
Jan 1, 2006
Aug 9, 2018
Jun 6, 2019
Dec 15, 2015
Jun 13, 2019
Feb 25, 2016
Apr 30, 2019
Jun 6, 2005
Jun 21, 2016
Mar 4, 2009
Dec 4, 2018
Oct 13, 2009
Aug 17, 2018
Oct 15, 2012
Feb 21, 2018
Sep 15, 2015
Feb 18, 2016
Feb 6, 2018
Aug 6, 2018
Oct 11, 2018
Jun 20, 2017
nan
nan
Mar 6, 2017
Aug 2, 2016
Mar 2020
Nov 23, 2018
Mar 3, 2017
Oct 10, 2016
Aug 26, 2016
Mar 21, 2018
Mar 5, 2008
May 30, 2019
Sep 3, 2019
Apr 28, 2015
Oct 2, 2018
Aug 2, 2016
Oct 30, 2018
Jul 10, 2011
May 5, 1994
Apr 19, 2017
Sep 22, 2015
Aug 15, 2018
Jun 16, 2017
Aug 29, 2017
Nov 16, 2018
Jun 27, 2013
May 1, 2015
Aug 22, 2016
Jan 31, 2019
Mar 28, 2019
Oct 17, 2017
Feb 26, 2016
Dec 16, 2015
Dec 7, 2015
Aug 21

Nov 20, 2001
Jun 29, 2010
May 2, 2019
Aug 29, 2017
Jul 5, 2016
May 31, 2018
Oct 18, 2018
Nov 8, 2018
Mar 4, 2011
Mar 24, 2003
Jul 20, 2017
Nov 3, 2016
Jul 2014
May 13, 2003
Mar 15, 2018
Feb 26, 2019
Apr 26, 2019
Jun 5, 2018
May 18, 2017
Jan 19, 2015
Jan 30, 2019
Apr 25, 2018
Mar 13, 2018
nan
Feb 18, 2019
Sep 20, 2018
Feb 27, 2017
Apr 23, 2015
Nov 19, 2015
Sep 22, 2015
May 14, 2015
Nov 17, 2015
Dec 20, 2017
Oct 4, 1996
Feb 28, 2018
Feb 12, 2019
Aug 17, 2018
Nov 18, 2016
Jun 6, 2014
Sep 23, 2014
Jun 6, 2017
May 20, 2019
Nov 3, 2015
Jul 19, 2017
Jun 6, 2018
Sep 10, 2004
Oct 15, 2015
Nov 17, 2016
Feb 1, 2018
Apr 18, 2019
Jun 5, 2019
Nov 17, 2016
Jan 31, 2019
Jul 28, 2010
Apr 25, 2017
Jul 9, 2018
Feb 7, 2019
Oct 9, 2018
Mar 5, 2015
Mar 9, 2016
Jun 7, 2016
Nov 1, 2017
Jan 7, 2003
Mar 8, 2011
Oct 12, 2010
Mar 12, 2018
Aug 7, 2018
Jun 11, 2019
Jun 18, 2018
Nov 6, 2018
Mar 24, 2016
Sep 27, 2017
Sep 14, 2018
Dec 22, 2017
Jun 25, 2015
Nov 2, 2017
Mar 18, 2019
Aug 1, 2017
Aug 8, 2017
Feb 21, 2018


Jul 12, 2005
Dec 19, 2016
May 29, 2019
Mar 7, 2018
Jan 22, 2018
Sep 28, 2015
Oct 31, 1996
Aug 22, 2017
Dec 10, 2015
May 1, 2019
Jun 5, 2007
Nov 20, 2015
Mar 29, 1996
Jul 13, 2018
Jan 8, 2016
Nov 24, 2009
Mar 21, 2014
Dec 12, 2006
Jul 3, 2015
Nov 6, 2015
Feb 7, 2017
Jun 6, 2019
Sep 21, 2017
Jan 25, 2019
Oct 11, 2011
Oct 11, 2018
Oct 14, 2016
Oct 29, 1997
Apr 5, 2016
Jul 10, 2015
Jun 21, 2016
Aug 6, 2014
Jun 21, 2018
Feb 25, 2014
May 29, 2015
May 20, 2015
Nov 16, 2011
Mar 28, 2017
Jan 7, 2019
May 21, 2019
Feb 28, 1998
Aug 7, 2017
Nov 22, 2016
Dec 15, 2011
Jan 21, 2019
Aug 28, 2015
Oct 21, 2013
Sep 19, 2018
Aug 30, 2016
May 30, 2019
Mar 15, 2018
Jun 30, 2017
Apr 2, 2015
Jun 1, 2015
Sep 13, 2018
May 8, 2018
Dec 8, 2018
Apr 4, 2019
Jul 6, 2017
Mar 30, 2017
Jul 1, 2013
Oct 17, 1997
Mar 26, 2019
Sep 23, 2010
Nov 3, 2011
Apr 13, 2016
Mar 8, 2018
Dec 12, 2014
Mar 1, 2017
Jul 25, 2017
Dec 14, 2017
Oct 30, 2014
Jun 21, 2018
Sep 21, 2018
Aug 25, 2016
Dec 19, 2016
Sep 28, 2017
Mar 15, 2001
Aug 31, 

Aug 28, 1998
Mar 8, 2018
Feb 1, 2019
Nov 13, 2001
May 25, 2017
May 13, 2019
Apr 21, 2016
Aug 11, 2016
Sep 13, 2018
Jun 30, 1995
Apr 26, 2019
Oct 30, 2012
Sep 20, 2017
Dec 24, 2018
Mar 29, 2011
Dec 7, 2005
Mar 22, 2019
Sep 14, 2017
Dec 17, 2018
Jan 23, 2014
Sep 21, 2018
Sep 20, 2013
Apr 11, 2017
May 12, 2017
Mar 3, 2017
Mar 3, 2016
Jan 5, 2018
Mar 14, 2011
Feb 22, 2019
Jan 18, 2018
Mar 31, 2017
Sep 26, 2018
Aug 1, 2006
Sep 29, 2017
Dec 9, 2014
Dec 18, 2018
May 11, 2015
Jan 30, 2018
May 30, 2013
Nov 30, 2016
Nov 21, 2016
Jun 15, 2018
Apr 18, 2017
Apr 28, 2019
Apr 21, 2017
Dec 19, 2018
Feb 27, 2018
Jun 7, 2016
Nov 30, 2017
Sep 10, 2018
Nov 6, 2014
Jun 5, 2019
Aug 21, 2018
May 8, 2015
Feb 6, 2017
Oct 2, 2017
Oct 24, 2013
Jun 10, 2013
Jan 23, 2019
Oct 7, 2014
Jan 7, 2019
Dec 1, 2015
May 8, 2014
Apr 5, 2016
Mar 28, 2017
Mar 6, 1998
Nov 2, 2010
Nov 9, 2018
Jan 31, 2019
Jan 11, 2018
Oct 17, 2006
Sep 21, 2018
Feb 3, 2015
Jun 17, 2016
May 8, 2014
Jul 25, 2017
Mar 24, 2016
Nov 16, 2016
Jun 3, 201

...

Dec 3, 2014
Jul 30, 2015
Aug 4, 2017
Mar 19, 2018
Jun 27, 2017
Aug 7, 2018
Mar 16, 2017
Mar 2, 2017
Jul 15, 2016
Nov 6, 2015
Apr 30, 2019
May 29, 2014
Sep 20, 2016
Apr 17, 2015
Nov 19, 2009
Mar 1, 2009
Apr 2, 2018
Dec 1, 2015
Dec 21, 2018
Mar 29, 2002
Jan 9, 2019
Nov 15, 2016
Jun 18, 2018
Nov 28, 2012
Dec 1, 2016
Sep 22, 2017
Jan 17, 2019
Apr 19, 2016
Oct 31, 2001
Mar 5, 2003
Mar 3, 2014
Dec 1, 2017
Jun 2, 2019
Jul 21, 2016
Aug 1, 2017
Feb 23, 2018
Sep 8, 2011
Nov 5, 2015
Feb 24, 2017
Mar 27, 2019
Nov 7, 2018
Nov 10, 2014
Jun 6, 2011
Dec 31, 2000
Sep 6, 2012
Aug 6, 2014
Sep 26, 2005
Nov 20, 2018
Jun 3, 2019
Aug 10, 2018
Nov 17, 2016
Apr 24, 2012
Dec 16, 2016
Apr 17, 2011
Feb 20, 2015
Nov 19, 2018
Sep 13, 2017
Aug 12, 2014
Oct 19, 2018
Jul 22, 2016
Apr 24, 2018
Apr 4, 2016
Jun 10, 2019
Oct 11, 2017
Sep 10, 2013
Jul 6, 2006
Nov 23, 2017
Dec 25, 2017
Mar 31, 1997
Dec 20, 2016
Sep 22, 2016
Apr 6, 2018
Jan 10, 2019
Jan 20, 2003
May 23, 2018
Dec 6, 2012
Apr 24, 2001
Aug 15, 1995
Jan 26, 2018

Jan 9, 2018
Apr 24, 2019
Oct 22, 2018
Jul 18, 2014
Dec 26, 2017
Jun 12, 2018
Nov 23, 2016
Jun 3, 2018
May 10, 2012
Jun 7, 2014
Jun 8, 2017
Jan 25, 2018
Jan 20, 2017
Jul 9, 2018
Sep 15, 2017
Dec 17, 2017
Nov 16, 2015
Dec 28, 2016
Nov 2, 2016
Apr 13, 2017
Dec 18, 2015
Mar 27, 2015
Mar 13, 2015
Sep 25, 2017
Jan 24, 2018
Nov 10, 2016
Jun 5, 2018
Sep 16, 2011
Jun 15, 2016
May 17, 2018
Aug 23, 2018
Oct 4, 2007
Jul 27, 2018
Aug 10, 2018
Sep 15, 2006
Dec 7, 2017
Sep 6, 2017
Jun 6, 2018
Nov 15, 2017
Oct 6, 2016
Apr 17, 2015
Mar 14, 2013
Apr 24, 2018
Sep 4, 2013
Jun 13, 2019
Nov 13, 2017
Aug 28, 2012
Sep 27, 2018
Mar 31, 2019
Apr 1, 2015
Oct 20, 2018
Jan 30, 2018
Dec 14, 2018
Oct 23, 2017
Oct 17, 2014
Aug 1, 2001
Oct 19, 2017
Jun 6, 2019
Nov 12, 2015
Apr 26, 2019
Jan 1, 2018
Nov 24, 2016
Nov 7, 2007
Mar 29, 2018
Jan 13, 2010
Sep 16, 2009
Oct 24, 2017
Apr 28, 2018
Mar 7, 2019
Oct 4, 2018
Apr 26, 2006
May 29, 2017
Oct 25, 2011
Oct 12, 2007
Mar 28, 2014
Apr 21, 2018
Nov 6, 2014
Dec 19, 2018
Nov 9, 

Jul 27, 2015
Jan 21, 2010
Mar 13, 2013
Mar 28, 2003
Jul 11, 2014
Aug 24, 2016
Mar 15, 2015
Oct 28, 2016
Apr 5, 2016
Sep 6, 2016
Oct 2, 2016
Dec 19, 2014
Nov 7, 2016
Aug 25, 2016
Jun 7, 2010
Sep 29, 2016
Jul 18, 2016
Oct 26, 2017
Nov 8, 2016
Jul 15, 2016
Aug 9, 2016
Jul 11, 2016
Nov 22, 1996
Oct 23, 2017
Jul 5, 2016
Jul 28, 2015
Feb 22, 2017
Oct 23, 2015
Aug 10, 2016
Jun 9, 2015
Jun 19, 2015
Mar 27, 2019
Apr 29, 2015
Jun 1, 2008
Aug 30, 2016
Aug 16, 2016
Aug 11, 2016
Oct 2, 2014
Sep 10, 2014
Aug 25, 2015
Jun 13, 2014
Oct 17, 2013
Jul 15, 2014
May 22, 2010
Sep 19, 2014
Oct 9, 2009
Jan 26, 2007
Sep 22, 2016
Mar 27, 2014
Jul 28, 2015
Feb 25, 2014
Apr 19, 2012
Dec 5, 2013
Oct 9, 2015
Dec 25, 2015
Aug 30, 2016
Oct 3, 2013
Sep 24, 2015
Nov 25, 2015
Apr 29, 2016
Jul 22, 2015
Sep 19, 2008
Aug 12, 2015
Aug 22, 2016
Apr 14, 2016
Aug 21, 2015
Jun 10, 2016
May 20, 2016
Jan 27, 2016
Mar 7, 2016
Jun 4, 2015
Jun 16, 2017
Feb 19, 2016
Oct 25, 2016
Oct 5, 2016
Sep 15, 2016
Jul 16, 2010
Jan 6, 2006
Oct 2

Dec 11, 2014
Dec 8, 2014
Feb 12, 2015
Feb 12, 2014
Mar 23, 2016
Mar 24, 2015
Oct 28, 2015
Aug 7, 2014
Jul 22, 2014
Jul 23, 2015
Aug 14, 2014
Jul 3, 2014
May 20, 2014
May 20, 2014
May 5, 2014
Jul 11, 2012
Aug 5, 2014
Feb 1, 2019
Mar 13, 2019
Feb 14, 2019
Apr 17, 2019
Dec 12, 2018
Nov 29, 2018
Nov 6, 2018
Jan 25, 2019
Nov 9, 2018
Oct 22, 2018
Jan 31, 2019
May 16, 2019
Apr 4, 2019
Feb 15, 2019
Mar 1, 2019
Aug 9, 2018
Oct 30, 2018
Feb 28, 2019
Aug 7, 2009
Dec 5, 2008
Jan 30, 2017
Feb 6, 2017
Feb 20, 2017
Feb 17, 2017
Mar 21, 2017
Apr 8, 2015
Jul 8, 2016
Jan 29, 2014
Mar 20, 2015
Apr 11, 2012
Mar 31, 2017
Oct 5, 2017
May 11, 2012
Apr 4, 2011
Aug 2, 2013
Sep 6, 2017
Aug 31, 2017
Nov 11, 2014
Sep 15, 2016
Mar 15, 2010
May 8, 2019
Oct 26, 2016
Aug 26, 2015
Dec 18, 2018
Mar 20, 2017
Jan 31, 2018
Apr 10, 2017
Apr 12, 2018
Nov 18, 2014
Jan 28, 2010
Sep 27, 2010
Mar 30, 2016
Sep 17, 2015
Nov 2, 2018
Sep 30, 2016
May 26, 2015
Sep 22, 2014
Oct 17, 2018
Aug 11, 2015
Aug 25, 2014
Sep 11, 2017
Oct 25, 

url  \
0       https://store.steampowered.com/app/379720/DOOM/   
1     https://store.steampowered.com/app/578080/PLAY...   
2     https://store.steampowered.com/app/637090/BATT...   
3       https://store.steampowered.com/app/221100/DayZ/   
4     https://store.steampowered.com/app/8500/EVE_On...   
...                                                 ...   
7995  https://store.steampowered.com/app/391040/F1_2...   
7996  https://store.steampowered.com/app/208500/F1_2...   
7997  https://store.steampowered.com/app/40930/The_M...   
7998  https://store.steampowered.com/app/360650/12_L...   
7999  https://store.steampowered.com/app/808080/Ulti...   

                                        name  release_date  \
0                                       DOOM  May 12, 2016   
1              PLAYERUNKNOWN'S BATTLEGROUNDS  Dec 21, 2017   
2                                 BATTLETECH  Apr 24, 2018   
3                                       DayZ  Dec 13, 2018   
4                                 EVE Online   May 6, 2003   
...                                      ...           ...   
7995                                 F1 2016  Aug 18, 2016   
7996                                F1 2012™  Sep 17, 2012   
7997  The Misadventures of P.B. Winterbottom  Apr 20, 2010   
7998  12 Labours of Hercules III: Girl Power   Aug 7, 2015   
7999                      Ultimate Racing 2D  May 24, 2018   

                                developer  \
0                             id Software   
1                        PUBG Corporation   
2                     Harebrained Schemes   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters,Feral Interactive (Mac)   
7996               Codemasters Birmingham   
7997                    The Odd Gentlemen   
7998       Jetdogs Studios,Zoom Out Games   
7999                          Applimazing   

                                publisher  \
0                      Bethesda Softworks   
1                        PUBG Corporation   
2                     Paradox Interactive   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters Feral Interactive (Mac)   
7996                          Codemasters   
7997                                   2K   
7998                      Jetdogs Studios   
7999                          Applimazing   

                                           popular_tags  \
0     FPS,Gore,Action,Demons,Shooter,First-Person,Gr...   
1     Survival,Shooter,Multiplayer,Battle Royale,PvP...   
2     Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...   
3     Survival,Zombies,Open World,Multiplayer,PvP,Ma...   
4     Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...   
...                                                 ...   
7995  Racing,Simulation,Sports,Multiplayer,Singlepla...   
7996  Racing,Simulation,Sports,Realistic,Multiplayer...   
7997  Indie,Puzzle,Platformer,Adventure,Time Manipul...   
7998  Casual,Strategy,Time Management,Singleplayer,M...   
7999         Racing,Simulation,Sports,Indie,Top-Down,2D   

                                           game_details  \
0     Single-player,Multi-player,Co-op,Steam Achieve...   
1                Multi-player,Online Multi-Player,Stats   
2     Single-player,Multi-player,Online Multi-Player...   
3     Multi-player,Online Multi-Player,Steam Worksho...   
4     Multi-player,Online Multi-Player,MMO,Co-op,Onl...   
...                                                 ...   
7995  Single-player,Multi-player,Steam Achievements,...   
7996  Single-player,Multi-player,Co-op,Steam Achieve...   
7997  Single-player,Steam Achievements,Partial Contr...   
7998  Single-player,Steam Achievements,Steam Trading...   
7999  Single-player,Multi-player,Online Multi-Player...   

                                              languages  \
0     English,French,Italian,German,Spa

In [23]:
import numpy as np

w30 = np.zeros(8000)
wAll = np.ones(8000)
for i in range(0, len(purged_steam_data)):
    age = purged_steam_data['age'][i]
    if (30 < age):
        w30[i] = ((age - 30)/30)
        
total = w30 + 1
w30 = w30/total
wAll = wAll/total

purged_steam_data['w30'] = w30
purged_steam_data['wAll'] = wAll

#keys for new columns:
#'w30' -> weight of the 30 days recommendations 
#'wAll' -> weight of all recommendations 

In [24]:
#
psd = purged_steam_data
purged_steam_data['weighted_revs'] = (psd['w30'] * (psd['rec_rev_score']/100)) + (psd['wAll'] * (psd['all_rev_score']/100)) 

purged_steam_data

url  \
0       https://store.steampowered.com/app/379720/DOOM/   
1     https://store.steampowered.com/app/578080/PLAY...   
2     https://store.steampowered.com/app/637090/BATT...   
3       https://store.steampowered.com/app/221100/DayZ/   
4     https://store.steampowered.com/app/8500/EVE_On...   
...                                                 ...   
7995  https://store.steampowered.com/app/391040/F1_2...   
7996  https://store.steampowered.com/app/208500/F1_2...   
7997  https://store.steampowered.com/app/40930/The_M...   
7998  https://store.steampowered.com/app/360650/12_L...   
7999  https://store.steampowered.com/app/808080/Ulti...   

                                        name  release_date  \
0                                       DOOM  May 12, 2016   
1              PLAYERUNKNOWN'S BATTLEGROUNDS  Dec 21, 2017   
2                                 BATTLETECH  Apr 24, 2018   
3                                       DayZ  Dec 13, 2018   
4                                 EVE Online   May 6, 2003   
...                                      ...           ...   
7995                                 F1 2016  Aug 18, 2016   
7996                                F1 2012™  Sep 17, 2012   
7997  The Misadventures of P.B. Winterbottom  Apr 20, 2010   
7998  12 Labours of Hercules III: Girl Power   Aug 7, 2015   
7999                      Ultimate Racing 2D  May 24, 2018   

                                developer  \
0                             id Software   
1                        PUBG Corporation   
2                     Harebrained Schemes   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters,Feral Interactive (Mac)   
7996               Codemasters Birmingham   
7997                    The Odd Gentlemen   
7998       Jetdogs Studios,Zoom Out Games   
7999                          Applimazing   

                                publisher  \
0                      Bethesda Softworks   
1                        PUBG Corporation   
2                     Paradox Interactive   
3                     Bohemia Interactive   
4                                     CCP   
...                                   ...   
7995  Codemasters Feral Interactive (Mac)   
7996                          Codemasters   
7997                                   2K   
7998                      Jetdogs Studios   
7999                          Applimazing   

                                           popular_tags  \
0     FPS,Gore,Action,Demons,Shooter,First-Person,Gr...   
1     Survival,Shooter,Multiplayer,Battle Royale,PvP...   
2     Mechs,Strategy,Turn-Based,Turn-Based Tactics,S...   
3     Survival,Zombies,Open World,Multiplayer,PvP,Ma...   
4     Space,Massively Multiplayer,Sci-fi,Sandbox,MMO...   
...                                                 ...   
7995  Racing,Simulation,Sports,Multiplayer,Singlepla...   
7996  Racing,Simulation,Sports,Realistic,Multiplayer...   
7997  Indie,Puzzle,Platformer,Adventure,Time Manipul...   
7998  Casual,Strategy,Time Management,Singleplayer,M...   
7999         Racing,Simulation,Sports,Indie,Top-Down,2D   

                                           game_details  \
0     Single-player,Multi-player,Co-op,Steam Achieve...   
1                Multi-player,Online Multi-Player,Stats   
2     Single-player,Multi-player,Online Multi-Player...   
3     Multi-player,Online Multi-Player,Steam Worksho...   
4     Multi-player,Online Multi-Player,MMO,Co-op,Onl...   
...                                                 ...   
7995  Single-player,Multi-player,Steam Achievements,...   
7996  Single-player,Multi-player,Co-op,Steam Achieve...   
7997  Single-player,Steam Achievements,Partial Contr...   
7998  Single-player,Steam Achievements,Steam Trading...   
7999  Single-player,Multi-player,Online Multi-Player...   

                                              languages  \
0     English,French,Italian,German,Spa

In [25]:
psd = purged_steam_data

weighted_results = {}
'''
for url in purged_steam_data['url']:
    games = []
    for game in results[url]:
        games.append((game[1], game[0] * psd.loc[psd['url'] == game[1]]['weighted_revs']))
    
    weighted_results[url] = games'''

similarities = cosine_similarities
for idx, row in purged_steam_data.iterrows():
   similar_indices = cosine_similarities[idx].argsort()[:-100:-1] #get top 99 most common games
   for i in similar_indices:
        similarities[idx][i] = cosine_similarities[idx][i] * purged_steam_data['weighted_revs'][i]
        
   similar_indices = similarities[idx].argsort()[:-100:-1]
   similar_items = [(similarities[idx][i], purged_steam_data['url'][i]) for i in similar_indices] 
   
   #similar_items.append((similarities.argsort(), purged_steam_data['url'][i]))
   #similar_items = [(cosine_similarities[idx][i] * purged_steam_data['weighted_revs'][i], purged_steam_data['url'][i]) for i in similar_indices] 
   weighted_results[row['url']] = similar_items[1:]
   


#print(psd.loc[psd['url'] == 'https://store.steampowered.com/app/379720/DOOM/']['weighted_revs'])
weighted_results['https://store.steampowered.com/app/379720/DOOM/']
#weighted_results['https://store.steampowered.com/app/379720/DOOM/']

[(0.6456717532797253,
  'https://store.steampowered.com/app/214870/Painkiller_Hell__Damnation/'),
 (0.5741719491385329, 'https://store.steampowered.com/app/519860/DUSK/'),
 (0.5379219370655134,
  'https://store.steampowered.com/app/13230/Unreal_Tournament_2004_Editors_Choice_Edition/'),
 (0.5356362048567533, 'https://store.steampowered.com/app/2290/Final_DOOM/'),
 (0.5296999641682331, 'https://store.steampowered.com/app/2320/QUAKE_II/'),
 (0.5151630434027682,
  'https://store.steampowered.com/app/2280/Ultimate_Doom/'),
 (0.5072013674238246,
  'https://store.steampowered.com/app/208200/Doom_3_BFG_Edition/'),
 (0.4842529001190972,
  'https://store.steampowered.com/app/422970/Devil_Daggers/'),
 (0.48297469544927246,
  'https://store.steampowered.com/app/41000/Serious_Sam_HD_The_First_Encounter/'),
 (0.4824939280509722,
  'https://store.steampowered.com/app/13210/Unreal_Tournament_3_Black/'),
 (0.4760279861172561, 'https://store.steampowered.com/app/2300/DOOM_II/'),
 (0.475679113091572,
  

In [26]:
results['https://store.steampowered.com/app/379720/DOOM/']

[(0.6883494171425641,
  'https://store.steampowered.com/app/214870/Painkiller_Hell__Damnation/'),
 (0.6697525813685312,
  'https://store.steampowered.com/app/782330/DOOM_Eternal/'),
 (0.6369681854912039,
  'https://store.steampowered.com/app/39530/Painkiller_Black_Edition/'),
 (0.6326916868878018,
  'https://store.steampowered.com/app/9070/DOOM_3_Resurrection_of_Evil/'),
 (0.6301346188358591, 'https://store.steampowered.com/app/2290/Final_DOOM/'),
 (0.5803456546652125, 'https://store.steampowered.com/app/731720/DOOMED/'),
 (0.5790904668500243, 'https://store.steampowered.com/app/9050/DOOM_3/'),
 (0.5752758900789829, 'https://store.steampowered.com/app/519860/DUSK/'),
 (0.5723276421746241, 'https://store.steampowered.com/app/2210/Quake_IV/'),
 (0.5709838169260986, 'https://store.steampowered.com/app/13520/Far_Cry/'),
 (0.565396011086472,
  'https://store.steampowered.com/app/378270/Putrefaction/'),
 (0.5641109393624519,
  'https://store.steampowered.com/app/208200/Doom_3_BFG_Edition/'),

In [27]:
weighted_results['https://store.steampowered.com/app/730/CounterStrike_Global_Offensive/']


[(0.7146942450959732, 'https://store.steampowered.com/app/222880/Insurgency/'),
 (0.6630789092135603,
  'https://store.steampowered.com/app/240/CounterStrike_Source/'),
 (0.6007706486653138,
  'https://store.steampowered.com/app/10180/Call_of_Duty_Modern_Warfare_2/'),
 (0.5423432341166335,
  'https://store.steampowered.com/app/440/Team_Fortress_2/'),
 (0.5416283796656179, 'https://store.steampowered.com/app/107410/Arma_3/'),
 (0.540791739171849, 'https://store.steampowered.com/app/500/Left_4_Dead/'),
 (0.5330210755672802,
  'https://store.steampowered.com/app/33930/Arma_2_Operation_Arrowhead/'),
 (0.5121983833918259, 'https://store.steampowered.com/app/242860/Verdun/'),
 (0.5119833852302945,
  'https://store.steampowered.com/app/35450/Red_Orchestra_2_Heroes_of_Stalingrad_with_Rising_Storm/'),
 (0.511138549381653,
  'https://store.steampowered.com/app/24240/PAYDAY_The_Heist/'),
 (0.5104746406529027,
  'https://store.steampowered.com/app/359550/Tom_Clancys_Rainbow_Six_Siege/'),
 (0.50913

In [45]:
def weighted_cos_recommend(library, no_games):
    recs = {} #recommendations
    for url in library:
        for game in weighted_results[url]:
            if game[1] in recs:
                recs[game[1]] += game[0]
            else: 
                recs[game[1]] = game[0]
            
    sorted_recs = sorted(recs.items() ,key=lambda x: x[1], reverse=True)
    sorted_recs = sorted_recs[:no_games]
    
    #for rec in sorted_recs:
    #    print("game " + rec[0])
    #    print("score " + str(rec[1]))
    
    return sorted_recs

library1 = ['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

library = ['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/',
           'https://store.steampowered.com/app/813820/Realm_Royale/',
           'https://store.steampowered.com/app/55230/Saints_Row_The_Third/']

weighted_cos_recommend(library, 10)
weighted_results['https://store.steampowered.com/app/489830/The_Elder_Scrolls_V_Skyrim_Special_Edition/']

[(0.6407356130731819,
  'https://store.steampowered.com/app/22330/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition/'),
 (0.6358034218000289,
  'https://store.steampowered.com/app/900883/The_Elder_Scrolls_IV_Oblivion_Game_of_the_Year_Edition_Deluxe/'),
 (0.5659736153746487,
  'https://store.steampowered.com/app/219780/Divinity_II_Developers_Cut/'),
 (0.560313149605899,
  'https://store.steampowered.com/app/22320/The_Elder_Scrolls_III_Morrowind_Game_of_the_Year_Edition/'),
 (0.5074807563855736, 'https://store.steampowered.com/app/551770/ECHO/'),
 (0.5058798658212461,
  'https://store.steampowered.com/app/611670/The_Elder_Scrolls_V_Skyrim_VR/'),
 (0.4882018451339289,
  'https://store.steampowered.com/app/288470/Fable_Anniversary/'),
 (0.4845112470010821,
  'https://store.steampowered.com/app/367500/Dragons_Dogma_Dark_Arisen/'),
 (0.45793410228541026, 'https://store.steampowered.com/app/40300/Risen/'),
 (0.4503485626288443,
  'https://store.steampowered.com/app/102500/Kingdoms_of_Ama

In [92]:
#evaluation
#create a list of libraries with 20 unique game urls
import random
random.seed(0)

libraries = []
for i in range(0, 100):
    libraries.append([])
    for j in range(0, 20):
        url = psd["url"][random.randint(0, 8000)]
        if 0 < libraries[i].count(url):
            --j
        else:
            libraries[i].append(url)

In [163]:
#peform recomendations
from sklearn.preprocessing import MinMaxScaler

cos_recs = []
weighted_cos_recs = []
for library in libraries:
    cos_recs.append(cos_recommend(library, 10))
    weighted_cos_recs.append(weighted_cos_recommend(library, 10))
    
for i in range(0, len(cos_recs)):
    cos_recs[i] = pd.DataFrame(np.array(cos_recs[i]), columns=['a', 'b'])['b']

for i in range(0, len(weighted_cos_recs)):
    weighted_cos_recs[i] = pd.DataFrame(np.array(weighted_cos_recs[i]), columns=['a', 'b'])['b']
#normalise recommendations
scaler = MinMaxScaler()
cos_recs = scaler.fit_transform(cos_recs)
weighted_cos_recs = scaler.fit_transform(weighted_cos_recs)


In [164]:
#customer opinion, 1 = would buy, 0 = wouldn't  buy
opinions = []
for i in range(0, 100):
    opinions.append([])
    for j in range(0, 10):
        opinions[i].append(random.randint(0,1))


In [165]:
from sklearn.metrics import mean_squared_error
from math import sqrt
    
#find RMSE
cos_RMSE = sqrt(mean_squared_error(opinions, cos_recs))
weighted_cos_RMSE =  sqrt(mean_squared_error(opinions, weighted_cos_recs))

#cos_RMSE 0.5850476088649076
#weighted_cos_RMSE 0.5865904706379691

In [175]:
#finding mean average precsion MAP
import ml_metrics 
#round recommendations to 1 or 0
cos_recs_rnd = np.empty(shape=(100, 10))
for i in range(0, len(cos_recs)):
    cos_recs_rnd[i] = list(map(round, cos_recs[i]))
    
weighted_cos_recs_rnd = np.empty(shape=(100, 10))
for i in range(0, len(weighted_cos_recs)):
    weighted_cos_recs_rnd[i] = list(map(round, weighted_cos_recs[i]))
    
print(ml_metrics.mapk(opinions, cos_recs_rnd, 100))
ml_metrics.mapk(opinions, weighted_cos_recs_rnd, 100)

0.10599999999999998